**Imports**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import pickle
import time
import missingno as msno
from statistics import mean
from collections import Counter, OrderedDict
from IPython.display import display
import import_ipynb
import UTILS as utils
from time import sleep

import FundamentalAnalysis as fa

importing Jupyter notebook from UTILS.ipynb


In [2]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
api_key = "Here would go my personal API key"

------------

**Load data**

*Files with data from year with displacement -X*

In [ ]:
file1 = 'DATA_ttm/S_P500_data_ttm_Xyeardisp/S_P500_dataframe_with_ratios_ttm_Xyeardisp.csv'
file2 = 'DATA_ttm/S_P500_data_ttm_Xyeardisp/S_P500_companies_financials_data_ttm_Xyeardisp.pickle'

In [ ]:
data_loaded = utils.data_loading(file1, file2)

In [ ]:
df = data_loaded[0]
dict_companies = data_loaded[1]

In [ ]:
df.head(2)

----------------

**Add sectors and industries info to the dataset**

In [ ]:
sectors_df, industries_df = utils.get_company_sectors_and_industries(df, dict_companies)

In [ ]:
if set(['sector','industry']).issubset(df.columns) == False:
    df.insert(0, "sector", sectors_df.sector)
    df.insert(1, "industry", industries_df.industry)

-------------------

**Compute intrinsic value**

NCAVPS = (Current Assets - Total Liabilities) / Number of Shares

In [ ]:
def compute_ncavps_instrinsic_value(dict_companies, ticker, year_disp=0):
    
    balance_sheet = dict_companies[ticker]['_Company__balance_sheet']
    current_assets = balance_sheet.loc['totalCurrentAssets'][4*year_disp:4+(4*year_disp)].mean()
    total_liabilities = balance_sheet.loc['totalLiabilities'][4*year_disp:4+(4*year_disp)].mean()
    enterprise_value =  dict_companies[ticker]['_Company__enterprise_value']
    try:
        number_shares = enterprise_value.loc['numberOfShares'][4*year_disp]
        date = balance_sheet.columns[4*year_disp]
    except:
        return "Error. No data for this ticker"
    
    if number_shares == 0:
        return 0, date
    
    intrinsic_value = (current_assets - total_liabilities)/number_shares
    
    return intrinsic_value, date

In [ ]:
def stock_price_intrinsic_value_comparator(dict_companies, ticker, intrinsic_value, date):
    
    stock_data = fa.stock_data(ticker, interval="1d")
    stock_data.index = pd.to_datetime(stock_data.index)
    
    format_date = pd.to_datetime(date+'-01')
    stock_price = stock_data.iloc[stock_data.index.get_loc(format_date, method='nearest')].close
        
    if stock_price == 0:
        pr_intrinsic_price = 0
    else:
        pr_intrinsic_price = intrinsic_value/stock_price #ALREADY CHANGED, DELETED THE TRAILING -1
    
    return pr_intrinsic_price

In [ ]:
ncavps_by_sector = dict()
sectors = utils.get_sectors_and_industries(df, dict_companies)[0]
for sector in sectors:
    sector_df = utils.get_sector_companies(df, dict_companies, sector)
    ncavps_by_sector[sector] = dict()
    for ticker in sector_df.index:
        intrinsic = compute_ncavps_instrinsic_value(dict_companies, ticker, year_disp=(X years of disp))
        if intrinsic == "Error. No data for this ticker":
            ncavps_by_sector[sector][ticker] = [np.nan]
            ncavps_by_sector[sector][ticker].append(np.nan)
        else:
            ncavps_by_sector[sector][ticker] = [intrinsic[0]]
            ncavps_by_sector[sector][ticker].append(stock_price_intrinsic_value_comparator(dict_companies, ticker, intrinsic[0], intrinsic[1]))

In [ ]:
ncavps_by_sector.keys()

In [ ]:
with open(f'models_outputs/NCAVPS_output/X_year_disp/ncavps_by_sector_Xyear.pickle', 'wb') as file:
    pickle.dump(ncavps_by_sector, file)

**Retrieve the saved dictionary**

In [ ]:
ncavps_by_sector_dict = dict()
with open('models_outputs/NCAVPS_output/X_year_disp/ncavps_by_sector_Xyear.pickle','rb') as file:
    raw_data = file.read()
    ncavps_by_sector_dict.update(pickle.loads(raw_data))

In [ ]:
ncavps_by_sector = dict()
for sector in ncavps_by_sector_dict.keys():
    ncavps_by_sector[sector] = pd.DataFrame.from_dict(ncavps_by_sector_dict[sector], orient='index', columns=['intrinsic_value', 'intrval_vs_currentprice'])

In [ ]:
ncavps_by_sector['Industrials']